In [1]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import warnings 
import pandas_ta
warnings.filterwarnings('ignore')
from datetime import timedelta

In [2]:
sp500=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol']=sp500['Symbol'].str.replace('.','-')
symbols_list=sp500['Symbol'].unique().tolist()
end_date='2024-08-15'
start_date=pd.to_datetime(end_date)-pd.DateOffset(365*8)
df=yf.download(tickers=symbols_list, start=start_date, end=end_date).stack()
df.index.names=['date','ticker']
df.columns=df.columns.str.lower()
df

[*********************100%%**********************]  503 of 503 completed


Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-08-17 A        45.481186   48.430000   48.630001   48.080002   48.630001   
           AAL      35.278728   36.599998   36.750000   36.189999   36.680000   
           AAPL     25.185986   27.305000   27.342501   27.084999   27.275000   
           ABBV     47.342251   66.830002   67.349998   66.349998   66.690002   
           ABT      38.596272   44.680000   45.139999   44.439999   45.139999   
...                       ...         ...         ...         ...         ...   
2024-08-14 XYL     131.490005  131.490005  132.229996  130.580002  131.720001   
           YUM     137.259995  137.259995  138.199997  137.050003  137.500000   
           ZBH     108.629997  108.629997  110.169998  107.760002  109.580002   
           ZBRA    333.239990  333.239990  333.660004  327.589996  332.230011   
           ZTS     183.380005  183.380005  188.500000  182.490005  188.050003   

Price                   volume  
date       ticker               
2016-08-17 A         3154900.0  
           AAL       8002000.0  
           AAPL    101424000.0  
           ABBV      7945400.0  
           ABT       8630300.0  
...                        ...  
2024-08-14 XYL        813400.0  
           YUM       1267900.0  
           ZBH       1166000.0  
           ZBRA       248800.0  
           ZTS       2153100.0  

[990557 rows x 6 columns]

In [ ]:
df1=df.copy()
df1_dropped=df1.drop(columns=['high','low','open','volume','close'])
df3=df1_dropped.unstack().reset_index()


In [9]:
def fetch_data(tickers, start_date, end_date):
    """Fetch historical data for the given tickers."""
    data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')
    return data

In [12]:
def calculate_trailing_returns(df, window=30):
    returns= pd.DataFrame()
    for ticker in df.columns.levels[0]:
        adj_close=df[ticker]['Adj Close']
        trailing_return=adj_close.pct_change(window).dropna()
        returns[ticker]=trailing_return
    returns.index=pd.to_datetime(returns.index)
    return returns   

In [30]:
#Weekly Positions
def weekly_positions(returns_df, top_n=5, bottom_n=5):
    returns_df = returns_df.resample('W').last()

    long_candidates={}
    short_candidates={}

    for date, data in returns_df.iterrows():
    # long_candidates= weekly_df.apply(lambda x: x.nsmallest(10).index.tolist(),axis=1)
    # short_candidates = weekly_df.apply(lambda x: x.nlargest(10).index.tolist(),axis=1)
    # return long_candidates,short_candidates
        long_candidates[date] = data.nsmallest(bottom_n).index.tolist()
        short_candidates[date] = data.nlargest(top_n).index.tolist()
    
    return long_candidates, short_candidates

In [31]:
def plot_candidates(data, candidates, position_type):
    for ticker in candidates:
        plt.figure(figsize=(10,5))
        data[ticker]['Adj Close'].plot(title=f"{position_type} Candidate: {ticker}")
        plt.xlabel('Date')
        plt.ylabel('Adjusted Close Price')
        plt.show()

In [32]:
from datetime import datetime, timedelta
sp500_tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA','AAL','AMD','IBM','ORCL','NKE','PEP']  # Example tickers
start_date = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')
end_date = datetime.now().strftime('%Y-%m-%d')

In [33]:
data=fetch_data(sp500_tickers, start_date, end_date)

[*********************100%%**********************]  11 of 11 completed


In [34]:
returns_df = calculate_trailing_returns(data, window=5)
returns_df

,AAL,AAPL,AMD,AMZN,GOOGL,IBM,MSFT,NKE,ORCL,PEP,TSLA
Date,,,,,,,,,,,
2023-08-24,-0.031894,0.013678,-0.025278,-0.015973,-0.001078,0.020546,0.009751,-0.070633,-0.017148,-0.007117,0.049357
2023-08-25,-0.030585,0.023612,-0.030346,0.000300,0.018986,0.027862,0.020539,-0.056960,-0.003435,0.006959,0.107198
2023-08-28,-0.028364,0.024738,-0.051839,-0.011434,0.020565,0.026286,0.005654,-0.031402,0.002144,0.018074,0.032601
2023-08-29,0.004720,0.038876,0.002461,0.004916,0.042532,0.035055,0.018452,0.003055,0.035267,0.029767,0.102877
2023-08-30,-0.000677,0.036053,-0.025953,-0.003321,0.026517,0.024057,0.005474,0.033924,0.027834,0.016618,0.084607
...,...,...,...,...,...,...,...,...,...,...,...
2024-08-09,0.023884,-0.016465,0.013359,-0.005718,-0.017941,0.021174,-0.006047,0.004459,-0.009154,-0.031734,-0.036934
2024-08-12,0.035560,0.040674,0.014464,0.035896,0.019089,0.042699,0.029508,0.045085,0.036915,-0.015054,-0.006989
2024-08-13,0.053706,0.068987,0.084114,0.051257,0.037084,0.031373,0.036035,0.082759,0.047188,-0.000696,0.035835


In [35]:
longs, shorts = weekly_positions(returns_df, top_n=5, bottom_n=5)

In [37]:
for date in sorted(longs.keys()):
    print(f"Week of {date.date()}:")
    print(f"Long Positions: {longs[date]}")
    print(f"Short Positions: {shorts[date]}")
    print()


Week of 2023-08-27:
Long Positions: ['NKE', 'AAL', 'AMD', 'ORCL', 'AMZN']
Short Positions: ['TSLA', 'IBM', 'AAPL', 'MSFT', 'GOOGL']

Week of 2023-09-03:
Long Positions: ['PEP', 'AAL', 'MSFT', 'IBM', 'TSLA']
Short Positions: ['AMD', 'AAPL', 'GOOGL', 'ORCL', 'NKE']

Week of 2023-09-10:
Long Positions: ['AAPL', 'AAL', 'TSLA', 'NKE', 'PEP']
Short Positions: ['ORCL', 'MSFT', 'IBM', 'AMD', 'AMZN']

Week of 2023-09-17:
Long Positions: ['ORCL', 'AAL', 'AMD', 'AAPL', 'NKE']
Short Positions: ['TSLA', 'PEP', 'AMZN', 'GOOGL', 'IBM']

Week of 2023-09-24:
Long Positions: ['TSLA', 'AMZN', 'NKE', 'AMD', 'GOOGL']
Short Positions: ['IBM', 'AAPL', 'PEP', 'AAL', 'MSFT']

Week of 2023-10-01:
Long Positions: ['IBM', 'PEP', 'ORCL', 'AAPL', 'AMZN']
Short Positions: ['AMD', 'NKE', 'TSLA', 'GOOGL', 'MSFT']

Week of 2023-10-08:
Long Positions: ['PEP', 'AAL', 'AMZN', 'IBM', 'NKE']
Short Positions: ['GOOGL', 'AMD', 'TSLA', 'ORCL', 'AAPL']

Week of 2023-10-15:
Long Positions: ['AAL', 'TSLA', 'IBM', 'AMD', 'ORCL']
S

In [40]:
# if not longs.empty() and not shorts.empty():
#     last_week = sorted(longs.keys())[-1]
#     plot_candidates(data, longs[last_week], "Long")
#     plot_candidates(data, shorts[last_week], "Short")

AttributeError: 'dict' object has no attribute 'empty'